In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
import os

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/My Drive/LLM e Psicometria/00 Validazione PFA/Analisi/Dark Triad')

In [6]:
!ls

cos_matrices  Dark_triad_items.gsheet  PFA_for_DarkTriad.ipynb


In [9]:
# Check and install the required packages
import subprocess
import sys

# Function to install packages
def install(package):
    if package not in sys.modules:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
packages = ['xlwt', 'sentence_transformers', 'factor-analyzer', 'scipy']

# Installing the required packages
for package in packages:
    install(package)

In [13]:
# Make gsheet readable
sheet_name = 'Dark_triad_items' # replace with your own sheet name
sheet_id = '1S_CsA-7MB-tmPAKagQuvpmEFc-S1-XGG5mK-IpG56gI' # replace with your sheet's ID
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [14]:
import pandas as pd
import numpy as np
import xlwt
# read in file with items text etc.
df_items = pd.read_csv(url)
df_items.head()

,Number,Factor,Item,Item simplified,Sign
0,1,Machiavellanism,Tendo a manipolare gli altri per ottenere ciò ...,manipolare gli altri per ottenere ciò che voglio,+
1,2,Machiavellanism,Ho usato l’inganno o la menzogna per ottenere...,usare l’inganno o la menzogna per ottenere ciò...,+
2,3,Machiavellanism,Ho usato l’adulazione per ottenere ciò che volevo,usare l’adulazione per ottenere ciò che volevo,+
3,4,Machiavellanism,Tendo a sfruttare gli altri per i miei scopi,sfruttare gli altri per i miei scopi,+
4,5,Psychopathy,Tendo a non provare rimorso,non provare rimorso,+


# Part I: Create embeddings for PFA

Below we do the folowing:
- Group item in a single list per facet
- Calculate item embeddings
- Reverse item embeddings (if necessary)
- Aggregate item embeddings to create facet embedding
- Calculate facet list embedding
- Compute cosine similarities
- Store results

In [15]:
# Calculate item embeddings
# Reverse item embeddings if necessary

# To make the code shorter and sightly more efficient we loop through the models that we are using in the paper.
# First we create a list of models (which all have italian in the language they can be used in according to huggingface)
models = ['nli-distilroberta-base-v2',
          'paraphrase-multilingual-mpnet-base-v2',
          'paraphrase-multilingual-MiniLM-L12-v2',
          'intfloat/multilingual-e5-base',
          'LaBSE',
          'dwulff/mpnet-personality'] #consider adding the finetuned model for psicometrista

# Import the necessary libraries and functions
from sentence_transformers import SentenceTransformer, util
from huggingface_hub import snapshot_download
import tensorflow_hub as hub

# Create an empty data frame, which we will then populate with the different type of embeddings
facet_embeddings_sentences = pd.DataFrame()

for mod in models:
  if mod == 'Dimitre/universal-sentence-encoder':
    item_embed = [] #create list for item-level embed
    item_embed_rev = [] #create list for item-level embed accounting for sign
    model_path = snapshot_download(repo_id = 'Dimitre/universal-sentence-encoder')
    model = hub.KerasLayer(handle = model_path)
      #encode items
    for item in range(0,len(df_items['Number'])): #loop over all the items
      item_embed.append(model([df_items['Item'].iloc[item]]).numpy().flatten())
      if df_items['Sign'].iloc[item][0] == '-': #if items is negatively keyed, reverse the embeddings
        item_embed_rev.append(model([df_items['Item'].iloc[item]]).numpy().flatten()*-1)
      else:
        item_embed_rev.append(model([df_items['Item'].iloc[item]]).numpy().flatten())
    df_items[mod + '_embeddings'] = item_embed #then, we append the two item-level embeddings list and give them a name based on the model we used
    df_items[mod + '_embeddings_rev'] = item_embed_rev
  else:
    model = SentenceTransformer(mod) #call the model
    item_embed = [] #create list for item-level embed
    item_embed_rev = [] #create list for item-level embed accounting for sign
    for item in range(0,len(df_items['Number'])): #loop over all the items
    #encode items
      item_embed.append(model.encode(df_items['Item'].iloc[item]))
      if df_items['Sign'].iloc[item][0] == '-': #if items is negatively keyed, reverse the embeddings
        item_embed_rev.append(model.encode(df_items['Item'].iloc[item])*-1)
      else:
        item_embed_rev.append(model.encode(df_items['Item'].iloc[item]))
    df_items[mod + '_embeddings'] = item_embed #then, we append the two item-level embeddings list and give them a name based on the model we used
    df_items[mod + '_embeddings_rev'] = item_embed_rev

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [16]:
df_items.columns

Index(['Number', 'Factor', 'Item', 'Item simplified', 'Sign',
       'nli-distilroberta-base-v2_embeddings',
       'nli-distilroberta-base-v2_embeddings_rev',
       'paraphrase-multilingual-mpnet-base-v2_embeddings',
       'paraphrase-multilingual-mpnet-base-v2_embeddings_rev',
       'paraphrase-multilingual-MiniLM-L12-v2_embeddings',
       'paraphrase-multilingual-MiniLM-L12-v2_embeddings_rev',
       'intfloat/multilingual-e5-base_embeddings',
       'intfloat/multilingual-e5-base_embeddings_rev', 'LaBSE_embeddings',
       'LaBSE_embeddings_rev', 'dwulff/mpnet-personality_embeddings',
       'dwulff/mpnet-personality_embeddings_rev'],
      dtype='object')

In [17]:
# To avoid having too long names for the output datsets, we create a list of names, which we will then use to save the embedding cosine matrices
# make sure that the names here are meaningful and aligned with those of the one in the cell above.
model_short = ['distilroberta', 'mpnet', 'miniLM', 'e5', 'labse', 'Wulff']

# Below, we loop over the different models we use for the study and compute the cosine sim. matrices.
for mod in range(0, len(models)):
  # create temporary empty lists for the item and one-pop method embeddings
  facet_embeddings_item = []
  facet_embeddings_item_rev = []

  #create cosine similarity matrix for each embedding calculation approach
  cosine_similarities_item = util.pytorch_cos_sim(df_items[models[mod] + '_embeddings'],df_items[models[mod] + '_embeddings']).numpy()
  cosine_similarities_item_rev = util.pytorch_cos_sim(df_items[models[mod] + '_embeddings_rev'],df_items[models[mod] + '_embeddings_rev']).numpy()
  #fill diagonal with 1. This is done to avoid efa functions reading the cosine matrix as covariance
  np.fill_diagonal(cosine_similarities_item,1)
  np.fill_diagonal(cosine_similarities_item_rev,1)

  #store results
  pd.DataFrame(cosine_similarities_item, columns = df_items['Item'].unique(), index = df_items['Item'].unique()).to_csv('./cos_matrices/matrix_concatenated_item_'+model_short[mod]+'.csv', index = False)
  pd.DataFrame(cosine_similarities_item_rev, columns = df_items['Item'].unique(), index = df_items['Item'].unique()).to_csv('./cos_matrices/matrix_concatenated_item_rev_'+model_short[mod]+'.csv', index = False)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/util.py:44: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  a = torch.tensor(a)
